In [4]:
import tensorflow as tf
import keras
import os
import cv2
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from skimage.metrics import structural_similarity as ssim
from keras.models import load_model, Model
from scipy.stats import gaussian_kde
from keras import Input
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.layers import Conv2D, MaxPooling2D, Resizing, UpSampling2D, Dense, BatchNormalization, LeakyReLU, Add, Flatten, Reshape, add
from keras.autoencoders import Sequential
from keras.optimizers import SGD, Adam
from keras.losses import MeanSquaredError, MeanAbsoluteError

In [ ]:
def generator_to_array(generator):
    # Initialiser une liste pour stocker les échantillons
    num_samples = len(generator)
    samples = []
    # Itérer sur le générateur pour obtenir les échantillons
    for i in range(num_samples):
        batch = generator.next()
        for image in batch[0]:
            samples.append(image)  # Ajouter uniquement les données (ignorer les étiquettes)
    return np.array(samples)

def mean_cosinus_similarity(v1, v2):
    cosinus_similarity = 0
    nb_individus = v1.shape[0]
    for i in range(nb_individus):
        cosinus_similarity += np.dot(v1[i], v2[i]) / (np.linalg.norm(v1[i]) * np.linalg.norm(v2[i]))
    return cosinus_similarity/nb_individus

def calculate_mean_dispersion(latent_representations):
    centroid = np.mean(latent_representations, axis=0)
    distance_squared = np.sum((latent_representations - centroid)**2, axis = 1)
    return np.mean(distance_squared)

def comparaison_visages(asian, asian_predict, white, white_predict, ethnie):

    plt.figure(figsize=(15, 6))

    # Affichage des images du premier trainset
    plt.subplot(2, 5, (1, 5))
    plt.axis("off")
    plt.title('Caucasiens')
    for i in range(5):
        plt.subplot(2, 5, i + 1)
        plt.imshow(white[i], cmap='gray')
        plt.axis('off')
        plt.subplot(2, 5, i + 6)
        plt.imshow(white_predict[i], cmap='gray')
        plt.axis('off')

    # Titre global
    plt.suptitle(f'Comparaison des images d entrées et de sorties caucasiennes d espace latent {ethnie}')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(15, 6))

    # Affichage des images du premier trainset
    plt.subplot(2, 5, (1, 5))
    plt.axis("off")
    plt.title('Asiatiques')
    for i in range(5):
        plt.subplot(2, 5, i + 1)
        plt.imshow(asian[i], cmap='gray')
        plt.axis('off')
        plt.subplot(2, 5, i + 6)
        plt.imshow(asian_predict[i], cmap='gray')
        plt.axis('off')

    # Titre global
    plt.suptitle('Comparaison des images d entrées et de sorties asiatiques')
    plt.tight_layout()
    plt.show()

def pca(latent_asian, latent_white, ethnie):
    pca = PCA(n_components=2)
    latent_pca_white = pca.fit_transform(latent_white)
    latent_pca_asian = pca.fit_transform(latent_asian)
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(latent_pca_white[:, 0], latent_pca_white[:, 1], color='r', alpha=0.5, label='white_predict')
    scatter_b = plt.scatter(latent_pca_asian[:, 0], latent_pca_asian[:, 1], color='b', alpha=0.5, label='asian_predict')
    plt.legend(handles=[scatter, scatter_b])
    plt.title(f'Projection ACP de l espace latent {ethnie}')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.show()

    return np.concatenate([latent_pca_white, latent_pca_asian], axis=0)


def cosinus_similarity(asian_flatten, asian_predict_flatten, white_flatten, white_predict_flatten):
    asian_cs = mean_cosinus_similarity(asian_flatten, asian_predict_flatten)
    white_cs = mean_cosinus_similarity(white_flatten, white_predict_flatten)
    print(f"Moyenne des similarités cosinus pour les individus caucasiens : {white_cs}")
    print(f"Moyenne des similarités cosinus pour les individus asiatiques : {asian_cs}")
    
def comparaison_dispersion(latent_asian, latent_white):
    asian_mean_dispersion = calculate_mean_dispersion(latent_asian)
    white_mean_dispersion = calculate_mean_dispersion(latent_white)
    print(f"Moyenne des dispersions des espaces latents pour les individus caucasiens : {white_mean_dispersion}")
    print(f"Moyenne des dispersions des espaces latents pour les individus asiatiques : {asian_mean_dispersion}")

def coef_bhattacharyya(latent_asian, latent_white):
    normalize_a = tf.nn.l2_normalize(np.mean(latent_asian, axis=0),axis=0)        
    normalize_b = tf.nn.l2_normalize(np.mean(latent_white, axis=0),axis=0)
    return tf.reduce_sum(tf.multiply(normalize_a,normalize_b))

def ssim_mean(ytrue, ypred, length):    
    res = 0
    for i in range(length):
        res+=ssim(ytrue[i], ypred[i], data_range=ytrue[i].max() - ytrue[i].min(), multichannel=False)
    
    return res/length

def structure_ssim(ytrue, ypred):
    return ssim(ytrue, ypred, data_range=ytrue.max() - ytrue.min(), multichannel=False)

In [ ]:
def create_autoencodeur(input_shape=(150, 150, 1), input_latent=512):

    np.random.seed(42)
    autoencoder=Sequential()

    autoencoder.add(Conv2D(64, (3,3),activation='relu',padding='same', input_shape=(input_shape)))
    autoencoder.add(MaxPooling2D((2,2), padding='same'))
    autoencoder.add(Conv2D(32, (3,3), activation='relu', padding='same'))
    autoencoder.add(MaxPooling2D((2,2), padding='same'))
    autoencoder.add(Conv2D(16, (3,3),activation='relu',padding='same'))
    autoencoder.add(MaxPooling2D((2,2), padding='same'))
    
    autoencoder.add(Dense(input_latent, activation='relu', name = 'latent_space'))

    autoencoder.add(Conv2D(16, (3,3), activation='relu', padding='same'))
    autoencoder.add(UpSampling2D((2,2)))
    autoencoder.add(Conv2D(32, (3,3), activation='relu', padding='same'))
    autoencoder.add(UpSampling2D((2,2)))
    autoencoder.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    autoencoder.add(UpSampling2D((2,2)))

    autoencoder.add(Conv2D(1, (3,3), activation='relu', padding='same'))

    autoencoder.add(Resizing(height=input_shape[0], width=input_shape[1], name='recon_image'))
    autoencoder.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

    autoencoder.summary()
    return autoencoder
autoencoder = create_autoencodeur(input_shape=(300, 300, 1), input_latent=2048)

In [ ]:
def create_autoencoder(input_shape=(150, 150, 1), input_latent=512):

    input_img = Input(shape=input_shape)
    filters = [16, 32, 64]
    autoencoder=Sequential()

    # Encoder
    for i, filter in enumerate(filters):
        autoencoder.add(Conv2D(filter, (3, 3), activation='relu', padding='same'))        
        autoencoder.add(LeakyReLU(alpha=0.1))
        autoencoder.add(BatchNormalization())
        autoencoder.add(MaxPooling2D((2, 2), padding='same'))
        print(f'After {i+1} Conv2D: {autoencoder.output_shape}')

    # Espace latent
    #x = Flatten()(x)
    autoencoder.add(Dense(input_latent, activation='relu', name='latent_space'))
    autoencoder.add(BatchNormalization(name='latent_space_norm'))

    # Décodeur
    for i, filter in enumerate(reversed(filters)):
        autoencoder.add(Conv2D(filter, (3, 3), activation='relu', padding='same'))        
        autoencoder.add(LeakyReLU(alpha=0.1))
        autoencoder.add(BatchNormalization())
        autoencoder.add(UpSampling2D((2, 2)))
        print(f'After {i+1} UpSampling2D: {autoencoder.output_shape}')
    
    autoencoder.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
    print(f'Decoded shape before resizing: {autoencoder.output_shape}')    
    # Resize to match input shape
    autoencoder.add(Resizing(height=input_shape[0], width=input_shape[1], name='final_output'))
    print(f'Decoded shape after resizing: {autoencoder.output_shape}')

    # Autoencoder autoencoder
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss=MeanAbsoluteError(), metrics=['accuracy'])
    
    # Display the model summary
    autoencoder.summary()
    
    return autoencoder

autoencoder = create_autoencoder(input_shape=(300, 300, 1), input_latent=2048)


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
trainset_white = train_datagen.flow_from_directory(
    '../../Datasets/STIM_NB_LumNorm/Train',
    classes=['Caucasiens'],
    target_size=(300, 300), 
    batch_size=6, 
    class_mode='input',
    color_mode='grayscale')
testset_white = train_datagen.flow_from_directory(
    '../../Datasets/STIM_NB_LumNorm/Test',
    classes=['Caucasiens'],
    target_size=(300, 300), 
    batch_size=6, 
    class_mode='input',
    color_mode='grayscale')
trainset_asian = train_datagen.flow_from_directory(
    '../../Datasets/STIM_NB_LumNorm/Train',
    classes=['Asiatiques'],
    target_size=(300, 300),
    batch_size=6,
    class_mode='input',
    color_mode='grayscale')
testset_asian = train_datagen.flow_from_directory(
    '../../Datasets/STIM_NB_LumNorm/Test',
    classes=['Asiatiques'],
    target_size=(300, 300),
    batch_size=6,
    class_mode='input',
    color_mode='grayscale')
trainset_mixed = train_datagen.flow_from_directory(
    '../../Datasets/STIM_NB_LumNorm/',
    classes=['Mixtes'],
    target_size=(300, 300), 
    batch_size=10, 
    class_mode='input',
    color_mode='grayscale')

### Entrainement réseau & Récupération de la couche latente

In [ ]:
model.fit(x=trainset_white, epochs=1000, validation_data=testset_white)

In [ ]:
pred = model.predict(testset_white)

In [ ]:
n = 6  # Number of images to display
plt.figure(figsize=(20, 4))

for i in range(n):
    # Afficher l'image originale
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(white[i].reshape(300, 300), cmap='gray')
    plt.title("Original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(pred[i].reshape(300, 300), cmap='gray')
    plt.title("Reconstruction")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
get_latent = keras.Model(inputs=model.input, outputs=model.get_layer('latent_space').output)

In [ ]:
latent_asian = get_latent.predict(trainset_asian, verbose=0)
latent_white = get_latent.predict(trainset_white, verbose=0)
latent_asian = latent_asian.reshape(50, -1)
latent_white = latent_white.reshape(50, -1)

test_latent_asian = get_latent.predict(testset_asian, verbose=0)
test_latent_white = get_latent.predict(testset_white, verbose=0)
test_latent_asian = test_latent_asian.reshape(50, -1)
test_latent_white = test_latent_white.reshape(50, -1)

asian_predict = model.predict(trainset_asian, verbose=0).reshape(50,300,300)
white_predict = model.predict(trainset_white, verbose=0).reshape(50,300,300)
asian = generator_to_array(trainset_asian).reshape(50,300,300)
white = generator_to_array(trainset_white).reshape(50,300,300)

asian_predict_flatten = asian_predict.reshape(50, -1)
white_predict_flatten = white_predict.reshape(50, -1)
asian_flatten = asian.reshape(50, -1)
white_flatten = white.reshape(50, -1)

test_asian = generator_to_array(testset_asian).reshape(6, 300, 300)
test_asian_predict = model.predict(testset_asian, verbose=0).reshape(6, 300, 300)
test_white = generator_to_array(testset_white).reshape(6, 300, 300)
test_white_predict = model.predict(testset_white, verbose=0).reshape(6, 300, 300)

test_asian_predict_flatten = test_asian_predict.reshape(50, -1)
test_white_predict_flatten = test_white_predict.reshape(50, -1)
test_asian_flatten = test_asian.reshape(50, -1)
test_white_flatten = test_white.reshape(50, -1)

In [ ]:
comparaison_visages(test_asian, test_asian_predict, test_white, test_white_predict, "white")
latent_pca = pca(latent_asian, latent_white, "white")
cosinus_similarity(asian_flatten, asian_predict_flatten, white_flatten, white_predict_flatten)
print(f"La similarité structurelle pour les caucasiens est de : {ssim_mean(white, white_predict, 50)}")
print(f"La similarité structurelle pour les asiatiques est de : {ssim_mean(asian, asian_predict, 50)}")
comparaison_dispersion(latent_asian, latent_white)